# F1 Data Engineering Project - Silver Layer
## Data Cleansing & Transformation

This notebook transforms Bronze layer data into clean, validated Silver layer tables.

**Transformations Applied:**
- Data type conversions (dates, numbers)
- Null value handling
- Column renaming for business context
- Creating derived columns
- Data quality improvements

## Configuration

In [0]:
# CONFIGURATION

CATALOG_NAME = "f1_dev"
BRONZE_SCHEMA = "bronze"
SILVER_SCHEMA = "silver"

# Full table references
bronze_db = f"{CATALOG_NAME}.{BRONZE_SCHEMA}"
silver_db = f"{CATALOG_NAME}.{SILVER_SCHEMA}"

print(f"Catalog: {CATALOG_NAME}")
print(f"Bronze Schema: {bronze_db}")
print(f"Silver Schema: {silver_db}")

Catalog: f1_dev
Bronze Schema: f1_dev.bronze
Silver Schema: f1_dev.silver


## Setup Unity Catalog

In [0]:
# Set catalog context
spark.sql(f"USE CATALOG {CATALOG_NAME}")

# Create Silver schema if not exists
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.{SILVER_SCHEMA}")
spark.sql(f"USE SCHEMA {SILVER_SCHEMA}")

print(f"Using: {CATALOG_NAME}.{SILVER_SCHEMA}")

Using: f1_dev.silver


## Import Libraries

In [0]:
from pyspark.sql.functions import (
    col, concat, lit, when, to_date, to_timestamp,
    current_timestamp, upper, lower, trim, regexp_replace,
    year, month, dayofmonth, split, coalesce
)
from pyspark.sql.types import IntegerType, DoubleType, DateType, TimestampType

## Transform Circuits

In [0]:
# Read Bronze circuits
circuits_bronze = spark.table(f"{bronze_db}.circuits")

# Transform circuits - add business-friendly names
circuits_silver = circuits_bronze \
    .select(
        col("circuit_id"),
        col("circuit_ref"),
        col("name").alias("circuit_name"),
        col("location").alias("circuit_location"),
        col("country").alias("circuit_country"),
        col("lat").alias("latitude"),
        col("lng").alias("longitude"),
        col("alt").alias("altitude_meters")
    ) \
    .withColumn("updated_at", current_timestamp())

# Write to Silver
circuits_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_db}.circuits")

print(f"[OK] circuits: {circuits_silver.count():,} records")

[OK] circuits: 77 records


## Transform Constructors

In [0]:
# Read Bronze constructors
constructors_bronze = spark.table(f"{bronze_db}.constructors")

# Transform constructors
constructors_silver = constructors_bronze \
    .select(
        col("constructor_id"),
        col("constructor_ref"),
        col("name").alias("constructor_name"),
        col("nationality").alias("constructor_nationality")
    ) \
    .withColumn("updated_at", current_timestamp())

# Write to Silver
constructors_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_db}.constructors")

print(f"[OK] constructors: {constructors_silver.count():,} records")

[OK] constructors: 212 records


## Transform Drivers

In [0]:
# Read Bronze drivers
drivers_bronze = spark.table(f"{bronze_db}.drivers")

# Transform drivers - add derived columns
drivers_silver = drivers_bronze \
    .select(
        col("driver_id"),
        col("driver_ref"),
        col("number").cast(IntegerType()).alias("driver_number"),
        col("code").alias("driver_code"),
        col("forename").alias("first_name"),
        col("surname").alias("last_name"),
        concat(col("forename"), lit(" "), col("surname")).alias("full_name"),
        to_date(col("dob"), "yyyy-MM-dd").alias("date_of_birth"),
        col("nationality").alias("driver_nationality")
    ) \
    .withColumn("updated_at", current_timestamp())

# Write to Silver
drivers_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_db}.drivers")

print(f"[OK] drivers: {drivers_silver.count():,} records")

[OK] drivers: 864 records


## Transform Races

In [0]:
# Read Bronze races
races_bronze = spark.table(f"{bronze_db}.races")

# Transform races
races_silver = races_bronze \
    .select(
        col("race_id"),
        col("year").alias("race_year"),
        col("round").alias("race_round"),
        col("circuit_id"),
        col("name").alias("race_name"),
        to_date(col("date"), "yyyy-MM-dd").alias("race_date"),
        col("time").alias("race_time"),
        to_date(col("quali_date"), "yyyy-MM-dd").alias("qualifying_date"),
        col("quali_time").alias("qualifying_time"),
        to_date(col("sprint_date"), "yyyy-MM-dd").alias("sprint_date"),
        col("sprint_time").alias("sprint_time")
    ) \
    .withColumn("race_datetime", 
                when(col("race_time").isNotNull(), 
                     to_timestamp(concat(col("race_date"), lit(" "), col("race_time"))))
                .otherwise(None)) \
    .withColumn("updated_at", current_timestamp())

# Write to Silver
races_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_db}.races")

print(f"[OK] races: {races_silver.count():,} records")

[OK] races: 1,149 records


## Transform Results

In [0]:
# Read Bronze results
results_bronze = spark.table(f"{bronze_db}.results")

# Transform results - add derived columns for analytics
results_silver = results_bronze \
    .select(
        col("result_id"),
        col("race_id"),
        col("driver_id"),
        col("constructor_id"),
        col("number").cast(IntegerType()).alias("driver_number"),
        col("grid").alias("grid_position"),
        col("position").cast(IntegerType()).alias("finish_position"),
        col("position_text"),
        col("position_order"),
        col("points").alias("points_earned"),
        col("laps").alias("laps_completed"),
        col("time").alias("finish_time"),
        col("milliseconds").cast(IntegerType()).alias("time_milliseconds"),
        col("fastest_lap").cast(IntegerType()).alias("fastest_lap_number"),
        col("rank").cast(IntegerType()).alias("fastest_lap_rank"),
        col("fastest_lap_time"),
        col("fastest_lap_speed").cast(DoubleType()).alias("fastest_lap_speed_kph"),
        col("status_id")
    ) \
    .withColumn("is_winner", when(col("finish_position") == 1, True).otherwise(False)) \
    .withColumn("is_podium", when(col("finish_position") <= 3, True).otherwise(False)) \
    .withColumn("is_points_finish", when(col("points_earned") > 0, True).otherwise(False)) \
    .withColumn("positions_gained", col("grid_position") - col("finish_position")) \
    .withColumn("updated_at", current_timestamp())

# Write to Silver
results_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_db}.results")

print(f"[OK] results: {results_silver.count():,} records")

[OK] results: 27,238 records


## Transform Qualifying

In [0]:
# Read Bronze qualifying
qualifying_bronze = spark.table(f"{bronze_db}.qualifying")

# Transform qualifying
qualifying_silver = qualifying_bronze \
    .select(
        col("qualify_id"),
        col("race_id"),
        col("driver_id"),
        col("constructor_id"),
        col("number").alias("driver_number"),
        col("position").alias("qualifying_position"),
        col("q1").alias("q1_time"),
        col("q2").alias("q2_time"),
        col("q3").alias("q3_time")
    ) \
    .withColumn("reached_q2", when(col("q2_time").isNotNull(), True).otherwise(False)) \
    .withColumn("reached_q3", when(col("q3_time").isNotNull(), True).otherwise(False)) \
    .withColumn("updated_at", current_timestamp())

# Write to Silver
qualifying_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_db}.qualifying")

print(f"[OK] qualifying: {qualifying_silver.count():,} records")

[OK] qualifying: 10,973 records


## Transform Lap Times

In [0]:
# Read Bronze lap times
lap_times_bronze = spark.table(f"{bronze_db}.lap_times")

# Transform lap times
lap_times_silver = lap_times_bronze \
    .select(
        col("race_id"),
        col("driver_id"),
        col("lap").alias("lap_number"),
        col("position"),
        col("time").alias("lap_time"),
        col("milliseconds").alias("lap_time_ms")
    ) \
    .withColumn("lap_time_seconds", col("lap_time_ms") / 1000.0) \
    .withColumn("updated_at", current_timestamp())

# Write to Silver
lap_times_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_db}.lap_times")

print(f"[OK] lap_times: {lap_times_silver.count():,} records")

[OK] lap_times: 615,738 records


## Transform Pit Stops

In [0]:
# Read Bronze pit stops
pit_stops_bronze = spark.table(f"{bronze_db}.pit_stops")

# Transform pit stops
pit_stops_silver = pit_stops_bronze \
    .select(
        col("race_id"),
        col("driver_id"),
        col("stop").alias("stop_number"),
        col("lap").alias("lap_number"),
        col("time").alias("time_of_day"),
        col("duration").alias("pit_duration"),
        col("milliseconds").alias("pit_duration_ms")
    ) \
    .withColumn("pit_duration_seconds", col("pit_duration_ms") / 1000.0) \
    .withColumn("updated_at", current_timestamp())

# Write to Silver
pit_stops_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_db}.pit_stops")

print(f"[OK] pit_stops: {pit_stops_silver.count():,} records")

[OK] pit_stops: 12,192 records


## Transform Driver Standings

In [0]:
# Read Bronze driver standings
driver_standings_bronze = spark.table(f"{bronze_db}.driver_standings")

# Transform driver standings
driver_standings_silver = driver_standings_bronze \
    .select(
        col("driver_standings_id"),
        col("race_id"),
        col("driver_id"),
        col("points").alias("championship_points"),
        col("position").alias("championship_position"),
        col("position_text"),
        col("wins").alias("total_wins")
    ) \
    .withColumn("updated_at", current_timestamp())

# Write to Silver
driver_standings_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_db}.driver_standings")

print(f"[OK] driver_standings: {driver_standings_silver.count():,} records")

[OK] driver_standings: 35,361 records


## Transform Constructor Standings

In [0]:
# Read Bronze constructor standings
constructor_standings_bronze = spark.table(f"{bronze_db}.constructor_standings")

# Transform constructor standings
constructor_standings_silver = constructor_standings_bronze \
    .select(
        col("constructor_standings_id"),
        col("race_id"),
        col("constructor_id"),
        col("points").alias("championship_points"),
        col("position").alias("championship_position"),
        col("position_text"),
        col("wins").alias("total_wins")
    ) \
    .withColumn("updated_at", current_timestamp())

# Write to Silver
constructor_standings_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_db}.constructor_standings")

print(f"[OK] constructor_standings: {constructor_standings_silver.count():,} records")

[OK] constructor_standings: 13,631 records


## Transform Status

In [0]:
# Read Bronze status
status_bronze = spark.table(f"{bronze_db}.status")

# Transform status - add status categories
status_silver = status_bronze \
    .select(
        col("status_id"),
        col("status").alias("status_description")
    ) \
    .withColumn("status_category",
                when(col("status_description") == "Finished", "Finished")
                .when(col("status_description").like("%Lap%"), "Lapped")
                .when(col("status_description").isin("Disqualified", "Excluded", "Not classified"), "Disqualified/NC")
                .when(col("status_description").isin("Accident", "Collision", "Collision damage", "Spun off"), "Accident")
                .when(col("status_description").like("%Engine%") | 
                      col("status_description").like("%Gearbox%") |
                      col("status_description").like("%Transmission%") |
                      col("status_description").like("%Hydraulic%") |
                      col("status_description").like("%Electrical%") |
                      col("status_description").like("%Brake%") |
                      col("status_description").like("%Suspension%"), "Mechanical")
                .when(col("status_description") == "Retired", "Retired")
                .otherwise("Other DNF")) \
    .withColumn("updated_at", current_timestamp())

# Write to Silver
status_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_db}.status")

print(f"[OK] status: {status_silver.count():,} records")

[OK] status: 139 records


## Transform Seasons

In [0]:
# Read Bronze seasons
seasons_bronze = spark.table(f"{bronze_db}.seasons")

# Transform seasons
seasons_silver = seasons_bronze \
    .select(
        col("year").alias("season_year"),
        col("url").alias("season_url")
    ) \
    .withColumn("updated_at", current_timestamp())

# Write to Silver
seasons_silver.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(f"{silver_db}.seasons")

print(f"[OK] seasons: {seasons_silver.count():,} records")

[OK] seasons: 76 records
